In [8]:
import numpy as np
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from PIL import Image #读取图像处理代码
import os.path
import shutil

Frist, create an array to record the pixels of each dot in one image
$$
  \begin{matrix}
   R & G & B \\
   256 & 0 & 0 \\
   34 & 68 & 68 \\
   . & . & . \\
   . & . & . \\
   . & . & . \\
  \end{matrix} 
$$

In [9]:
def image_to_array(img_file):
    im = img_file
    col, row = im.size
    data = np.zeros((row * col, 5))
    pixels = im.load()
    for i in range(row):
        for j in range(col):
            r, g, b = pixels[i, j]
            data[i * col + j, :] = r, g, b, i, j
    return data

Then, transform the RGB_array[color coloumn] above to a list with the length of $256 \times 265=65536$.  
For example data['red'] to a red_list with length of 65536  
And get the red_list of all $n$ images we got, forming an array with the shape of $n\times 65536$  
Do the same procedure to green and blue

In [10]:
def array_on_color():
    R = []
    G = []
    B = []
    path = './images/full1/'
    for filename in os.listdir(path):
        im = Image.open(os.path.join(path, filename))
        data = image_to_array(im)
        R.append(data[:, 0])
        G.append(data[:, 1])
        B.append(data[:, 2])
    red_arr = np.array(R).reshape(len(R), 256*256)
    green_arr = np.array(G).reshape(len(G), 256*256)
    blue_arr = np.array(B).reshape(len(B), 256*256)
    return red_arr, green_arr, blue_arr

Run PCA to each color array, got the frist PCA score of each image for each color. And finally use three PCA score of different colors to represent a picture.  
Like:
$$
\begin{matrix}
   R & G & B \\
   13.0 & 20.0 & 180.5 \\
   34.7 & 99.3 & 200.3 \\
   . & . & . \\
   . & . & . \\
   . & . & . \\
\end{matrix}
$$

In [11]:
def pca_on_color():
    pca_score_list = []
    for x in array_on_color():
        pca = PCA(n_components=100)
        x_pca = pca.fit_transform(x)
        pca_score_list.append(x_pca)
    print(pca_score_list)
    pca_score_arr = np.array(pca_score_list)[:, :, 0]
    print(pca_score_arr)
    print(pca_score_arr.shape)
    return pca_score_arr.T

Run kmeans to cluster the images based on their pca_arr, which is created by the above function.  
Store the cluster result in csv for further use.

In [12]:
def kmeans_on_pca_score():
    # pca_score_arr = pca_on_color()
    X = np.random.rand(360, 3)
    kmeans = KMeans(n_clusters=5, random_state=0).fit(X)
    df = pd.DataFrame({'r': X[:, 0], 'g': X[:, 1], 'b': X[:, 2], 'label': kmeans.labels_})
    df1 = pd.DataFrame({'idx': df.index, 'group_label': df['label']})
    path = './images/group_result.csv'
    df1.to_csv(path, index=False)

Pick a group of image, divding the images in this group into tranning_set and test_set for CNN modeling and further validation.

In [18]:
def get_image(groupid=0):
    image_list = pd.DataFrame(os.listdir('./images/full1/'))
    path = './images/group_result.csv'
    df = pd.read_csv(path)
    loc = df[df['group_label'] == groupid]['idx'].values
    selected_list = np.array(image_list.loc[loc, :])[:, 0]
    X = []
    src_dir = './images/full1/'
    train_des_dir = './images/grouped/train'+'_'+str(groupid)
    if not os.path.exists(train_des_dir):
        os.makedirs(train_des_dir)
        for filename in selected_list:
            shutil.copy(os.path.join(src_dir, filename), train_des_dir)
        return
    else:
        

def load_train_test_data():
    for i in range(5):
        get_image(groupid=i)
        print (i)
    print ("all image loaded")


In [19]:
load_train_test_data()

0
1
2
3
4
all image loaded
